<a href="https://colab.research.google.com/github/sushant822/machine-learning-challenge/blob/main/big_data_challenge_video_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os

# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.7'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [20]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-01 18:20:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.40MB/s    in 0.6s    

2020-11-01 18:20:33 (1.40 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [21]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://classdb27oct2020.s3-us-west-1.amazonaws.com/amazon_reviews_us_Digital_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
digital_video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
digital_video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [22]:
print(f"Original count: {digital_video_games_df.count()}")

Original count: 145431
Count after dropna: 145425
Count after dropping duplicates: 145425


In [23]:
digital_video_games_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [24]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = digital_video_games_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDHZ0IP0QWF3G|   39005147|B00SLM597S|     500010339| 2015-08-31|
|R2AQD17WE4BPXV|   10726509|B009IR1SR0|     912779847| 2015-08-20|
|R27V3DHADX5APY|    1709090|B00F4CEHNK|     341969535| 2015-08-19|
|R2Q0PGNE0M3ES4|     167253|B00TUEOSMS|      47086315| 2015-08-18|
|R1LB0DS6OZCIMW|   10534940|B00TUEOSMS|      47086315| 2015-08-17|
|R3HX7QICC0NOCK|   11267480|B00GAC1D2G|     384246568| 2015-08-15|
|R1KXTQXBE0CRQM|   22661511|B00T8NURF8|     181827382| 2015-08-15|
|R39TG3LST9JDA8|   18404333|B002VBWIP6|     232803743| 2015-08-10|
|R1W2UR1JVB2C13|   22353312|B010R8DUFQ|     473214709| 2015-08-09|
|R1M0DM7FCXI1H9|     535447|B00GAC1D2G|     384246568| 2015-08-06|
|R2IJD4JDICI4AF|   10733761|B009IR1SR0|     912779847| 2015-08-01|
| RWVPFDHF1M46B|    2585643|B00GAC1D2G|     384246568| 2015-07

In [25]:
products_df = digital_video_games_df.select(["product_id", "product_title"]).drop_duplicates()

In [26]:
reviews_df = digital_video_games_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
| RDHZ0IP0QWF3G|Maybe you'll have...|Crashed and never...|
|R2AQD17WE4BPXV|Can't play any of...|Can't play any of...|
|R27V3DHADX5APY|            One Star|Can't use to pay ...|
|R2Q0PGNE0M3ES4|This game has so ...|Story line was im...|
|R1LB0DS6OZCIMW|          Five Stars|Perfect. Now I ca...|
|R3HX7QICC0NOCK|            One Star|I don't appreciat...|
|R1KXTQXBE0CRQM|This is a great b...|This is a great b...|
|R39TG3LST9JDA8|          Five Stars|Good price saved 10$|
|R1W2UR1JVB2C13|Simplistic, but e...|This game is thor...|
|R1M0DM7FCXI1H9|          Five Stars|              Great.|
|R2IJD4JDICI4AF|Great games but h...|The games were gr...|
| RWVPFDHF1M46B|          Five Stars|                  (:|
|R3VRDDUP2Y8GLL|Enjoying this in ...|I'm a RPG guy fro...|
| RTQ74WPYINEQ4|      No Downloading|Amazon gives the ..

In [28]:
customers_df = digital_video_games_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47505356|             1|
|   21198249|            10|
|   11646226|             4|
|   50712554|             1|
|   13490955|             2|
|    2099668|             1|
|   29206489|             1|
|   14231016|             1|
|   46351928|             7|
|   28665190|             1|
|   44172434|             1|
|   45096613|             1|
|   13713446|             8|
|    2863501|             1|
|   39449017|             3|
|   21281723|             5|
|    3953141|             2|
|    8213600|             2|
|   33185883|             1|
|   15109931|             1|
+-----------+--------------+
only showing top 20 rows



In [29]:
vine_df = digital_video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDHZ0IP0QWF3G|          1|            1|          1|   N|
|R2AQD17WE4BPXV|          1|            1|          1|   N|
|R27V3DHADX5APY|          1|            0|          0|   N|
|R2Q0PGNE0M3ES4|          5|            0|          0|   N|
|R1LB0DS6OZCIMW|          5|            0|          0|   N|
|R3HX7QICC0NOCK|          1|            0|          0|   N|
|R1KXTQXBE0CRQM|          5|            0|          0|   N|
|R39TG3LST9JDA8|          5|            0|          0|   N|
|R1W2UR1JVB2C13|          5|            1|          1|   N|
|R1M0DM7FCXI1H9|          5|            0|          0|   N|
|R2IJD4JDICI4AF|          1|            2|          2|   N|
| RWVPFDHF1M46B|          5|            0|          0|   N|
|R3VRDDUP2Y8GLL|          5|            0|          0|   N|
| RTQ74WPYINEQ4|          1|            

**Saving data to AWS-RDS Database**

In [33]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ct26r4oxbudr.us-west-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "12345678", "driver":"org.postgresql.Driver"}

In [34]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [35]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [36]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [37]:
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)